**<center><font size="7">This is just preview for tutorial purpose</font></center>**

**Complete code is in "modelGenerator.py" folder and also in given <font size="4"> [link](https://github.com/frommindtopage/sklearn-machine-learning-model-generator/blob/main/modelGenerator.py)**

if you get error like "<font color="red">No unique mode; found 2 equally common values</font>"<br>
**then use latest version of python**

**<center><font size="6">class</font></center>**
**<font size="4" color="red">Use it only for sklearn model (scikit-learn)<font>**

In [1]:
class machineLearningModel:
    def optimize(this,data,label,typ='train'):
        from numpy import nan,array,where
        from pandas import DataFrame
        from statistics import mode
        if(typ == 'train'):
            code=dict()
            data = data.copy()
            __pipe = dict()
            for column in  data.columns:
                if(data[column].dtype == 'O'):
                    code[column] = dict()
                    index_list = array(sorted(data[column].value_counts().index))
                    for value in index_list:
                        if(value not in  [nan]+this.missing):
                            ind = where(index_list == value)[0][0]
                            code[column][value] = ind
                            data[column] = data[column].replace(value,ind)
                for miss in this.missing:
                    data[column] = data[column].replace(miss,nan)
                __pipe[column] = {'mean':data[column].mean(),"std":data[column].std(),"median":data[column].median(),"zero":0,'mode':mode(data[column])}
                data[column] = data[column].replace(nan,__pipe[column][this.replace[column]])
                if(column != this.label):
                    data[column] = (data[column]-__pipe[column]["mean"])/__pipe[column]["std"]
            this.__pipe=__pipe
            dataFeature = data.drop(this.label,axis = 1) 
            dataLabel = data[this.label].copy()
            return dataFeature,dataLabel,code
        elif(typ == 'test'):
            data = data.copy()
            for column in data.columns:
                if(data[column].dtype == 'O'):
                    for value in set(data[column]):
                        if(value not in [nan]+this.missing):
                            ind = this.code[column][value]
                            data[column] = data[column].replace(value,ind)
                for miss in this.missing+[nan]:
                    data[column] = data[column].replace(miss,this.__pipe[column][this.replace[column]])
                if(column != this.label):
                    data[column] = (data[column]-this.__pipe[column]["mean"])/this.__pipe[column]["std"]
            if(label != False):
                data,label = data.drop(this.label,axis = 1) ,data[this.label].copy()
            return data,label
        elif(typ == 'prediction'):
            df = DataFrame([data],columns=list(this.__pipe.keys())[:-1])
            return this.optimize(df,label=False,typ='test')
    
    def optimizePipline(this,feature,label = None,typ='train'):
        from numpy import nan
        if (typ == 'train'):
            dataFeature,dataLabel,code = this.optimize(feature,label)
            return dataFeature,dataLabel,code
        elif(typ == 'test'):
            dataFeature,dataLabel = this.optimize(feature,label,typ='test')
            return dataFeature,dataLabel
        elif(typ == 'prediction'):
            dataFeature,label = this.optimize(feature,False,typ)
            return dataFeature
        
    def __repr__(this):
        return this.desc
    def predict(this,feature,getCode=False):
        def labelRecogniser(code,getCode):
            if this.label in this.code and not getCode:
                return list(this.code[this.label].keys())[code]
            else:
                return code
        if(len(feature)==1):
            return labelRecogniser(this.__model.predict(this.optimizePipline(feature[0],label=None,typ='prediction').to_numpy())[0],getCode=getCode)
        else:
            return [labelRecogniser(this.__model.predict(this.optimizePipline(sampleFeature,label=None,typ='prediction').to_numpy())[0],getCode=getCode) for sampleFeature in feature]
    def performance(this):
        print(this)
        print("f1 score                      ",this.f1Score)
        print("precision score               ",this.precisionScore)
        print("recall score                  ",this.recallScore)
        print("confusion matrix              ",this.confusionMatrix.tolist())
        print("averge time per prediction    ",this.speed,"sec")
    def __init__(this,model,trainFeature,label,testFeature,desc=None,replace="median",missing=['?']):
        this.desc = desc
        this.label = label
        this.missing=missing
        if(type(replace)==str):
            replace=[replace]*len(trainFeature.columns)
        this.replace = {trainFeature.columns[column] : replace[column] for column in range(len(trainFeature.columns))}
        def trainingAndTesting(model,trainFeature,label,testFeature):
            from sklearn.metrics import mean_squared_error,precision_score, recall_score,confusion_matrix,f1_score
            from sklearn.model_selection import cross_val_score,cross_val_predict
            from numpy import sqrt
            from time import time
            trainFeature,trainLabel,this.code = this.optimizePipline(trainFeature,label)
            
            this.value = {this.code[this.label][x]:x  for x in this.code[this.label]}
            model.fit(trainFeature,trainLabel)
            this.__model = model
            ##speed calculation
            ini=time()
            this.predict(testFeature.to_numpy()[:1000,:-1])
            this.speed = (time()-ini)/1000
            ###################
            testFeature,testLabel = this.optimizePipline(testFeature,label,typ = 'test')
            predict = this.__model.predict(testFeature)
            error = sqrt(mean_squared_error(testLabel, predict))
            this.RMSE = error
            this.confusionMatrix = confusion_matrix(testLabel,predict)
            this.precisionScore = precision_score(testLabel, predict)
            this.recallScore = recall_score(testLabel,predict)
            this.f1Score = f1_score(testLabel, predict)
        trainingAndTesting(model,trainFeature,label,testFeature)

**<center><font size="7">Example</font></center>**

### importing dataset for traning and testing

In [2]:
from pandas import read_csv
testData,trainData = read_csv("data/adult test.csv"),read_csv("data/adult data.csv")

### importing model

### creating model

* ___model___
<br>machine learning model/function <br>
    _Example._ KNeighborsClassifier,MLPClassifier,SVC,GaussianProcessClassifier
* ___trainFeature___
<br>training dataset in panda dataframe form
* ___Label___<br>
column you want to predict<br>
    _Example._ i want to predict "income"
* ___trainFeature___
<br>testing dataset in panda dataframe form
* __desc__
<br>it is description of tha object.


In [3]:
from sklearn.neighbors import KNeighborsClassifier
kn = machineLearningModel(model=KNeighborsClassifier(),trainFeature=trainData,label="income",testFeature=testData,desc="KNeighborsClassifier model")

### model info

In [4]:
print(kn)
print("f1 score                      ",kn.f1Score)
print("precision score               ",kn.precisionScore)
print("recall score                  ",kn.recallScore)
print("confusion matrix              ",kn.confusionMatrix.tolist())
print("averge time per prediction    ",kn.speed,"sec")

KNeighborsClassifier model
f1 score                       0.6171712982165077
precision score                0.6589902568644819
recall score                   0.5803432137285491
confusion matrix               [[11280, 1155], [1614, 2232]]
averge time per prediction     0.046747205972671506 sec


In [5]:
kn.performance()

KNeighborsClassifier model
f1 score                       0.6171712982165077
precision score                0.6589902568644819
recall score                   0.5803432137285491
confusion matrix               [[11280, 1155], [1614, 2232]]
averge time per prediction     0.046747205972671506 sec


### prediction
<font color='red'>input should be 2 dimensional</font>

In [6]:
kn.predict([[25, 'Private', 226802, '11th', 7, 'Never-married','Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 0, 40,'United-States']])

'<=50K'

In [7]:
kn.predict([[25, 'Private', 226802, '11th', 7, 'Never-married','Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 0, 40,'United-States'],
       [38, 'Private', 89814, 'HS-grad', 9, 'Married-civ-spouse','Farming-fishing', 'Husband', 'White', 'Male', 0, 0, 50,'United-States'],
       [28, 'Local-gov', 336951, 'Assoc-acdm', 12, 'Married-civ-spouse','Protective-serv', 'Husband', 'White', 'Male', 0, 0, 40,'United-States'],
       [44, 'Private', 160323, 'Some-college', 10, 'Married-civ-spouse','Machine-op-inspct', 'Husband', 'Black', 'Male', 7688, 0, 40,'United-States'],
       [18, '?', 103497, 'Some-college', 10, 'Never-married', '?','Own-child', 'White', 'Female', 0, 0, 30, 'United-States']])

['<=50K', '<=50K', '<=50K', '>50K', '<=50K']

### if you want code 
each label has a inbuilt-code you can get it just using getCode = True

In [8]:
kn.predict([[25, 'Private', 226802, '11th', 7, 'Never-married','Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 0, 40,'United-States']],getCode=True)

0

You can check the value of above code using kn.value<br>
and 0 means '<=50k'

In [9]:
kn.value

{0: '<=50K', 1: '>50K'}

## missing datapoint

### missing

missing is a parameter <br>
**defult value: ['?']**<br>
this a list<br>
model treat all the element of that list as Nan

In [10]:
from sklearn.neighbors import KNeighborsClassifier
kn = machineLearningModel(model=KNeighborsClassifier(),trainFeature=trainData,label="income",testFeature=testData,desc="KNeighborsClassifier model",missing=['?','-'])

so this model treat '?' and '-' as Nan

### replace

missing is a parameter <br>
**defult value: "median"**<br>
this a list<br>
this tells what you want to replace in place of Nan

In [11]:
from sklearn.neighbors import KNeighborsClassifier
kn = machineLearningModel(model = KNeighborsClassifier(),trainFeature = trainData,label = "income",testFeature = testData,desc = "KNeighborsClassifier model",missing = ['?','-'],replace = 'mean')

this model will replace all Nan from all columns by mean() of that column

If you want to treat Nan differently in different column

In [12]:
from sklearn.neighbors import KNeighborsClassifier
kn = machineLearningModel(model = KNeighborsClassifier(),trainFeature = trainData,label = "income",testFeature = testData,desc = "KNeighborsClassifier model",missing = ['?','-'],replace = ["mean","median","zero","mean","median","zero","mean","median","zero","mode","median","zero","mean","median","zero"])

First element of <font size="3"> ___replace___</font> is _"mean"_&nbsp;&nbsp; means this model replace all the Nan in column1 by _mean_ of column1<br>
Second element of <font size="3"> ___replace___</font> is _"median"_&nbsp;&nbsp; means this model replace all the Nan in column2 by _median_ of column2<br>
and so on<br>
- you have 5 option which you can use in replace
    - **mean** :- this replace Nan by mean of the column
    - **median** :- this replace Nan by median of the column
    - **mode** :- this replace Nan by mode of the column
    - **zero** :- this replace Nan by zero
    - **std** :- this replace Nan by standard deviation of the column

**<center><font size="6">Saving the model for future</font></center>**

### Saving the model to file

In [13]:
from joblib import dump, load
from sklearn.neighbors import KNeighborsClassifier
kn = machineLearningModel(model=KNeighborsClassifier(),trainFeature=trainData,label="income",testFeature=testData,desc="KNeighborsClassifier model")
dump(kn, 'income prediction.joblib')

['income prediction.joblib']

Now you will get a file __'income prediction.joblib'__<br>
your model is saved in that file

### using model from file

You can import it anytime and you don't need to retrain it

In [14]:
from joblib import dump, load
kn = load('income prediction.joblib')

directly using the model for prediction

In [15]:
kn.predict([[44, 'Private', 160323, 'Some-college', 10, 'Married-civ-spouse','Machine-op-inspct', 'Husband', 'Black', 'Male', 7688, 0, 40,'United-States']])

'>50K'

In [16]:
kn.f1Score

0.6171712982165077

# I used KNeighborsClassifier but you can try others too
## Some of them are given below.

_#from sklearn.neighbors import KNeighborsClassifier_<br>
_#from sklearn.neural_network import MLPClassifier_<br>
_#from sklearn.svm import SVC_<br>
_#from sklearn.gaussian_process import GaussianProcessClassifier_<br>
_#from sklearn.tree import DecisionTreeClassifier_<br>
_#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier_

<font color="red" size="4">SVC and GaussianProcessClassifier takes a lot of time in training<br>
Sometimes my pc hangs</font>